In [255]:
import numpy as np
import math
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

results = []
with open('BR/BSG150_results.txt', 'r') as f:
    line = f.readline()
    cnt = 1
    while line:
        results.append(float(line))
        line = f.readline()
    
data = []
for k in range(16):
    with open('BR/BR'+str(k)+'.txt', 'r') as f:
        nb_instances = int(next(f))
        for i in range(nb_instances):
            id, random = [int(x) for x in next(f).split()]
            L,W,H = [int(x) for x in next(f).split()]
            nb_boxes = int(next(f))
            volumes = []; bases=[]; ratios=[]; perimeters=[]; ws = []; ls =[]; hs = []
            total_volume = 0
            nb_boxes2=0
            r1_total=0
            r2_total=0
            for i in range(nb_boxes):
                id_box, l, r1, w, r2, h, r3, n = [int(x) for x in next(f).split()]
                nb_boxes2 +=n
                for j in range(n):
                    #como obtener porcentaje de volumen ocupado por x% de cajas mas grandes
                    
                    volumes.append(l*w*h) 
                    perimeters.append(l+w+h)
                    bases.append(l*w)
                    ratios.append((h/l))
                    ws.append(w), hs.append(h), ls.append(l)
                    total_volume += l*w*h
                    r1_total += r1
                    r2_total += r2
            volumes.sort(reverse=True)
            box_quartile = []
            vtot = 0.0; m=0; perc=0.10
            for v in volumes:
                m += 1
                vtot += v
                if vtot>perc*total_volume: 
                    box_quartile.append(m/nb_boxes2)
                    perc += 0.5
                if vtot>perc*total_volume: 
                    box_quartile.append(m/nb_boxes2)
                    perc += 0.5
                if perc > 0.95: break
            #print(box_quartile)
                
                #if vtot>0.5*total_volume/(L*W*H): box_quartile.append(m/nb_boxes2)
                #if vtot>0.85*total_volume/(L*W*H): box_quartile.append(m/nb_boxes2)
                    

            #print(np.percentile(volumes, [25,50,75])/(L*W*H))
            
            data.append(np.concatenate((np.percentile(volumes, [5])/(L*W*H),
                                        np.percentile(perimeters, [5])/(L+W+H),
                                        np.percentile(bases, [5])/(L*W),
                                        np.percentile(ratios, [10,90]),
                                        #np.percentile(ls, [10])/L,
                                        #np.percentile(ws, [10])/W,
                                        np.percentile(hs, [5])/H,
                                        box_quartile,
                                       [r2_total/nb_boxes2,k/16])))
            #for p in range(3):
            #    data[-1][p]=math.pow(data[-1][p],1/2)
            

#print(data)
x = np.array(data)
y = np.array(results)
x = sm.add_constant(x)

#model = LinearRegression().fit(x, y)
model = sm.OLS(y, x).fit()
#r_sq = model.score(x, y)



#print('coefficient of determination:', r_sq)

#print(model.predict([[0,0.01,0.15,np.log(10)]])
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     216.5
Date:                Mon, 08 Jun 2020   Prob (F-statistic):          4.31e-288
Time:                        21:44:25   Log-Likelihood:                 4503.2
No. Observations:                1600   AIC:                            -8984.
Df Residuals:                    1589   BIC:                            -8925.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0804      0.007     11.761      0.000       0.067       0.094
x1             1.8138      0.661      2.744      0.006       0.517       3.110
x2            -0.5521      0.061     -9.026      0.000      -0.672      -0.432
x3             1.8824      0.183     10.262      0.000       1.523       2.242
x4             0.0493      0.008      6.041      0.000       0.033       0.065
x5            -0.0141      0.005     -3.001      0.003      -0.023      -0.005
x6             0.1682      0.036      4.664      0.000       0.097       0.239
x7             0.2827      0.060      4.737      0.000       0.166       0.400
x8            -0.0112      0.012     -0.925      0.355      -0.035       0.013
x9            -0.0669      0.003    -19.442      0.000      -0.074      -0.060
x10            0.0324      0.002     17.157      0.000       0.029       0.036
==============================================================================
Omnibus:                     1644.691   Durbin-Watson:                   1.695
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           229713.800
Skew:                           4.603   Prob(JB):                         0.00
Kurtosis:                      60.974   Cond. No.                     3.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""